In [1]:
import csv
import json
import pandas as pd
import re
import logging
import os
import random
import sys
from heapq import nlargest
import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd


with open('../ScientificWritingDataset/authorJsonAndPaperJson/author_json.json', 'r',encoding='utf-8') as f:
    authors_sampled = json.load(f)
print(len(authors_sampled))

with open('../ScientificWritingDataset/authorJsonAndPaperJson/paper_json.json', 'r',encoding='utf-8') as f:
    papers_sampled = json.load(f)
print(len(papers_sampled))


281
107


In [5]:
author_interests = pd.read_csv('../naive_method_v3.0/author_interests.tsv', sep='\t', header=None, names=["authorId","interests"])
author2interests = {}
small_interests = 0
small_interests_author_list = []

for i in range(len(author_interests["authorId"])):
    authorId = author_interests["authorId"][i]
    interests = json.loads(author_interests["interests"][i].replace("'","\""))
    if len(interests) <= 2:
        small_interests = small_interests + 1
        small_interests_author_list.append(authorId)
    author2interests[author_interests["authorId"][i]] = author_interests["interests"][i]

small_interests

0

In [10]:
error_list=[]
i = 0
openai.api_key = ""
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/")
]

    # prompt_construction v1.6
def construct_prompt_cot_agg(author_dict,authorId):
    prompt_str = ""

    question_example = []
    for index,item in enumerate(author_dict):
        if index < 10:
            question_example.append("Paper #"+str(index)+"\nAbstract: "+item["abstract"]+"\nTitle: "+item["title"])

    prompt_str = prompt_str + "I give you some titles and paper abstracts that you've written. Please summarize your top 3 research interests.\n"
    prompt_str = prompt_str + "\n".join(question_example) + "\n"
    prompt_str = prompt_str + "Please summarize top three key words that best represent your research interests in the format of \"['interest_1', 'interest_2', 'interest_3']\":\n"
    prompt_str = prompt_str + "Interests: "

    return prompt_str

def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

author2interests_llm = {}

for indice,authorId in enumerate(small_interests_author_list):
    prompt = construct_prompt_cot_agg(authors_sampled[str(authorId)]["papers"],authorId)
    result = run_openai(prompt,key_bundles,i,indice)
    print(indice, result)
    author2interests_llm[authorId]= result
 

In [6]:
author2interests = {}
for i in range(len(author_interests["authorId"])):
    authorId = author_interests["authorId"][i]
    interests = json.loads(author_interests["interests"][i].replace("'","\""))
    if len(interests) <= 2:
        author2interests[str(author_interests["authorId"][i])] = json.loads(author2interests_llm[authorId].replace("'","\""))
    else:
        author2interests[str(author_interests["authorId"][i])] = interests[:3]

with open("../naive_method_v1.0/author_interests.tsv", "wb", buffering=0) as out_file:
    for authorId in author2interests:
        write_str = bytes(str(authorId)+"\t"+str(author2interests[authorId])+"\n", 'utf-8')
        out_file.write(write_str)

In [7]:
openai.api_key = ""
openai.api_base = "https://biocodeeval-openai.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('', "https://biocodeeval-openai.openai.azure.com/"),
    ('', "https://biocodeeval-openai.openai.azure.com/"),
    ('', "https://biocodeeval-openai.openai.azure.com/")
]

    # prompt_construction v1.6
def construct_prompt_task_1(author_dict,authorId):
    prompt_str = ""

    question_example = []
    for index,item in enumerate(author_dict):
        if index < 15:
            question_example.append("Paper #"+str(index)+"\nAbstract: "+item["abstract"]+"\nTitle: "+item["title"])

    prompt_str = prompt_str + "You will be given some science papers that you've written, each of them contains a title and a abstract that you've written. Please summarize your top 3 research interests.\n"
    prompt_str = prompt_str + "\n".join(question_example) + "\n"
    prompt_str = prompt_str + "Please summarize top three key words that best represent your research interests in the format of \"['interest_1', 'interest_2', 'interest_3']\":\n"
    prompt_str = prompt_str + "Interests: "

    return prompt_str

def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo-16k", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ') 
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo-16k", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
 
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo-16k", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i) 
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i) 
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

id2interest_pred = {}
with open("author_interests_prediction_v1.0.tsv", "wb", buffering=0) as out_file:
    for i,authorId in enumerate(author2interests):
        prompt = construct_prompt_task_1(authors_sampled[authorId]["papers"],authorId)
        result = run_openai(prompt,key_bundles,i,i)
        id2interest_pred[authorId] = json.loads(result.replace("'","\""))
        print(i, result)
        write_str = bytes(str(authorId)+"\t"+result+"\n", 'utf-8')

0 ['Social media influence', 'Luxury brand marketing', 'AI robustness and explainability']
1 ['Internet of Things', 'Software assistants', 'Model-driven engineering']
2 ['Digital Twins', 'Model-Driven Engineering', 'Software Engineering']
3 ['Regression testing', 'Software process improvement', 'Evidence-based decision-making']
4 ['Automated testing', 'Test case prioritisation', 'Causal analysis of empirical data']
5 ['Ethnomedicinal investigation', 'Product customization', 'Software product line variability']
6 ['Software Product Line Engineering', 'Systematic Mapping Studies', 'Software Architecture Evaluation']
7 ['Blockchain technology', 'Smart Contracts', 'Software engineering']
8 ['Blockchain technology', 'Smart contracts', 'Software engineering']
9 ['Smart Contracts', 'Blockchain Technology', 'Software Engineering']
10 ['Blockchain software systems', 'Technical debt in open-source projects', 'Application of blockchain technology']
11 ['Bitcoin-based decentralised applications', 

In [8]:

from rouge import Rouge
import evaluate
def postprocess_text_generation(preds, labels):
    preds = [" ".join(pred).replace("\"","").strip() for pred in preds]
    labels = [[" ".join(label).strip()] for label in labels]
    return preds, labels

def create_metric_rouge():
    rouge_metric = evaluate.load('rouge')
    def compute_metrics(decoded_preds, decoded_labels):
        decoded_preds, decoded_labels = postprocess_text_generation(decoded_preds, decoded_labels)
        result_rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"rouge-1" : result_rouge["rouge1"], "rouge-L" : result_rouge["rougeL"]}
        return result
    return compute_metrics
pred = []
gold = []
for i in id2interest_pred:
    if i in authors_sampled and i in id2interest_pred:
        pred.append(id2interest_pred[i])
        gold.append(author2interests[i])
# print(pred)
metric = create_metric_rouge()
print(metric(pred, gold))
# results

2023-10-07 06:21:49.682057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 06:22:00.521794: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-07 06:22:00.522164: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-07 06:22:00.522199: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

{'rouge-1': 0.2559858557662884, 'rouge-L': 0.2299960483212423}


In [ ]:
error_list=[]
i = 0
openai.api_key = ""
openai.api_base = "https://biocodeeval-openai.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('', "https://biocodeeval-openai.openai.azure.com/"),
    ('', "https://biocodeeval-openai.openai.azure.com/"),
    ('', "https://biocodeeval-openai.openai.azure.com/")
]


def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ') 
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ') 
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ') 
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i) 
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i) 
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

# G-EVAL 
def construct_prompt_g_eval(source,output,dimension):

    Name = ""
    Description = ""
    Instruction = "Evaluation Steps:\n"
    dimensions_single_inputs = ["novelty","informativeness"]

    if dimension == "coherence":
        Name = "Coherence"
        Description = "Coherence (1-5) - the collective quality of all sentences. We align this dimension with the DUC quality question of structure and coherence whereby \"the title should be well-structured and well-organized. The title should not just be a heap of related information, but should build from sentence to a coherent body of information about a topic.\"\n"
        Instruction = Instruction + "1. Read the science paper abstract carefully and identify the main topic and key points." + "\n"
        Instruction = Instruction + "2. Read the title and compare it to the science paper abstract. Check if the title covers the main topic and key points of the paper abstract, and if it presents them in a clear and logical order." + "\n"
        Instruction = Instruction + "3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria." + "\n"
    elif dimension == "consistency":
        Name = "Consistency"
        Description = "Consistency (1-5) - the factual alignment between the title and the science paper abstract. A factually consistent title contains only statements that are entailed by the source document. Annotators were also asked to penalize titles that contained hallucinated facts. \n"
        Instruction = Instruction + "1. Read the science paper abstract carefully and identify the main facts and details it presents." + "\n"
        Instruction = Instruction + "2. Read the title and compare it to the source abstract. Check if the title contains any factual errors that are not supported by the abstract." + "\n"
        Instruction = Instruction + "3. Assign a score for consistency on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria." + "\n"
    elif dimension == "fluency":
        Name = "Fluency (1-3)"
        Description = "Fluency (1-3): the quality of the title in terms of grammar, spelling, punctuation, word choice, and sentence structure.\n"
        Description = Description + "- 1: Poor. The title has many errors that make it hard to understand or sound unnatural.\n"
        Description = Description + "- 2: Fair. The title has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.\n"
        Description = Description + "- 3: Good. The title has few or no errors and is easy to read and follow.\n"
        Instruction = ""
    elif dimension == "relevance":
        Name = "Relevance"
        Description = "Relevance (1-5) - selection of important content from the source. The title should include only important information from the source document. Annotators were instructed to penalize titles which contained redundancies and excess information.\n"
        Instruction = Instruction + "1. Read the title and the source document carefully." + "\n"
        Instruction = Instruction + "2. Compare the title to the source document and identify the main points of the science paper." + "\n"
        Instruction = Instruction + "3. Assess how well the title covers the main points of the article, and how much irrelevant or redundant information it contains." + "\n"
        Instruction = Instruction + "4. Assign a relevance score from 1 to 5." + "\n"
    
    elif dimension == "informativeness":
        Name = "Informativeness"
        Description = "Informativeness(1-5) - the amount of information, including concepts, key points and findings, from the title. Annotators were instructed to penalize titles which contained under-explored concepts or redundancies and excess information.\n"
        Instruction = Instruction + "1. Read the title and the source title carefully." + "\n"
        Instruction = Instruction + "2. Check if the title effectively communicate the main topic, key points, and significant findings or conclusions. Assess if the information provided is comprehensive, accurate, and complete." + "\n"
        Instruction = Instruction + "3. A good informative title (scored 4 or 5) should convey enough amount of conpcets and key points in a progressive manner, while the relatively poorer ones have vague descriptions and under-explored ideas(scored 1 or 2)." + "\n"
        Instruction = Instruction + "4. Assign a score for informativeness on a scale of 1 to 5, to make the reuslts more clearly distributed, please avoid giving score 4" + "\n"
        
    elif dimension == "novelty":
        Name = "Novelty (1-3)"
        Description = "Novelty (1-3): the uniqueness and originality of the title in terms of concept, perspective, and creativity. \n"
        Description = Description + "- 1: Poor. The title lacks originality and uniqueness, and it closely resembles existing or common titles. \n"
        Description = Description + "- 2: Fair. Thetitle shows some degree of uniqueness and creativity, but it still has elements that are common or predictable. \n"
        Description = Description + "- 3: Good. The title is highly original and unique, offering a fresh and creative perspective that stands out from common titles.\n"
        Instruction = "Assign a score for novelty on a scale of 1 to 3"

    prompt_str = ""
    prompt_str = prompt_str + "You will be given one title written for a science paper.\n"
    prompt_str = prompt_str + "Your task is to rate the output on one metric.\n"
    prompt_str = prompt_str + "Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.\n"
    prompt_str = prompt_str + "Evaluation Criteria:\n"
    prompt_str = prompt_str + Description
    prompt_str = prompt_str + Instruction
    if dimension not in dimensions_single_inputs:
        prompt_str = prompt_str + "Main topic:\n"
        prompt_str = prompt_str + source + "\n"
    prompt_str = prompt_str + "title:\n"
    prompt_str = prompt_str + output + "\n"
    prompt_str = prompt_str + "Evaluation Form (scores ONLY):\n"
    prompt_str = prompt_str + "- "+Name+":"

    return prompt_str

dimensions = ["novelty"]
geval_result = {}


i = 0
for j,id in enumerate(author2interests):
    geval_result[id] = {}
    if True:
        if id in authors_sampled and id in id2interest_pred:
            for dimension in dimensions:
                prompt = construct_prompt_g_eval(" ".join(author2interests[id])," ".join(id2interest_pred[id]),dimension)
                # print(prompt)
                
                # prompt = construct_prompt_g_eval(papers_sampled[id]["title"],papers_sampled[id]["abstract"],dimension)
                # break
                result = run_openai(prompt,key_bundles,i,i)
                print(result)
                print(str(j),str(dimension),str(result))
                geval_result[id][dimension] = result
                i = i + 1
        # break
        
geval_avg_scores = {"informativeness":0,"novelty":0}
count = 0
for paperId in geval_result:
    if "informativeness" in geval_result[paperId]:
        count = count + 1
        for dimension in dimensions:
            if "1. Fluency (1-3): " in geval_result[paperId][dimension]:
                geval_result[paperId][dimension].replace("geval_result[paperId][dimension]","")
            score = int(geval_result[paperId][dimension][0])
            geval_avg_scores[dimension] = geval_avg_scores[dimension] + score

for d in geval_avg_scores:
    print(d+ "\t" + str(geval_avg_scores[d]/count))
count

In [16]:
geval_avg_scores = {"informativeness":0,"novelty":0}
count = 0
for paperId in geval_result:
    if "informativeness" in geval_result[paperId]:
        count = count + 1
        for dimension in dimensions:
            if "1. Fluency (1-3): " in geval_result[paperId][dimension]:
                geval_result[paperId][dimension].replace("geval_result[paperId][dimension]","")
            try:
                score = int(geval_result[paperId][dimension][0])
                geval_avg_scores[dimension] = geval_avg_scores[dimension] + score
            except Exception as e:
                continue

for d in geval_avg_scores:
    print(d+ "\t" + str(geval_avg_scores[d]/count))
count

informativeness	3.3122171945701355
novelty	2.3936651583710407


221